In [4]:
import pandas as pd

# Replace 'file_path.csv' with the actual path to your CSV file
df = pd.read_csv('./csv/Posts.csv')

# Display the first few rows of the DataFrame
print(df.columns)


Index(['Id', 'OwnerUserId', 'LastEditorUserId', 'PostTypeId',
       'AcceptedAnswerId', 'Score', 'ParentId', 'ViewCount', 'AnswerCount',
       'CommentCount', 'OwnerDisplayName', 'LastEditorDisplayName', 'Title',
       'Tags', 'ContentLicense', 'Body', 'FavoriteCount', 'CreationDate',
       'CommunityOwnedDate', 'ClosedDate', 'LastEditDate', 'LastActivityDate'],
      dtype='object')
